### Importing necessary libararies

In [ ]:
import json
import glob
import numpy as np
import cv2
import copy
import torch
import torchvision
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.utils.data.dataset import Dataset
import os
import matplotlib.pyplot as plt
import face_recognition
from tqdm.autonotebook import tqdm

### Finding Frames

In [ ]:
# finding the number of frames for each video in dataset.
video_files = glob.glob('/Users/karthiksagar/DeepFake-Detection/FF++/real/*.mp4')
video_files1 = glob.glob('/Users/karthiksagar/DeepFake-Detection/FF++/fake/*.mp4')
video_files2 = glob.glob('/Users/karthiksagar/DeepFake-Detection/Celeb-DF/Celeb-real/*.mp4')
video_files3 = glob.glob('/Users/karthiksagar/DeepFake-Detection/Celeb-DF/Celeb-synthesis/*.mp4')
video_files4 = glob.glob('/Users/karthiksagar/DeepFake-Detection/Celeb-DF/YouTube-real/*.mp4')
video_files5 = glob.glob('/Users/karthiksagar/DeepFake-Detection/dfdc/*.mp4')

video_files = video_files+video_files1+video_files2+video_files3+video_files4+video_files5

frame_count = []
for video_file in video_files:
    cap = cv2.VideoCapture(video_file)
    if (int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) < 150):
        video_files.remove(video_file)
        continue
    frame_count.append(int(cap.get(cv2.CAP_PROP_FRAME_COUNT)))

print("Frames : ", frame_count)
print("Total Number of Videos : ", len(frame_count))
print("Average frames per video : ", np.mean(frame_count))

### DFDC-Dataset has average frames of 300
### Celeb-DF Dataset has average frames of 407
### Face Forensics FF++ Dataset has average frames of 790

### EXTRACTING FACE CROPPED IMAGES FROM VIDEOS

In [ ]:
import cv2
import os
import face_recognition

# Define the paths to the folders containing the videos
videos_folder_1 = "/Users/karthiksagar/Deep-Fake-Detection/FF++/fake"
videos_folder_2 = "/Users/karthiksagar/Deep-Fake-Detection/FF++/real"

# Define the path to the output folder for face-only videos
output_folder1 = "/Users/karthiksagar/Deep-Fake-Detection/FF++/Face-Image-Fake"
output_folder2 = "/Users/karthiksagar/Deep-Fake-Detection/FF++/Face-Image-Real"

# Function to extract faces from a video and save them as images
def extract_faces_from_video(video_path, output_folder):
    # Create output folder if it doesn't exist
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Open the video file
    video_capture = cv2.VideoCapture(video_path)
    frame_count = 0
    frames_processed = 0

    # Read until video is completed
    while frame_count < 5:
        # Read a single frame
        ret, frame = video_capture.read()
        if not ret:
            break  # Break the loop if no more frames are available

        # Convert frame to RGB (face_recognition library requires RGB format)
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Find faces in the frame
        face_locations = face_recognition.face_locations(rgb_frame)

        # If faces are found, extract and save them
        for top, right, bottom, left in face_locations:
            # Extract the face region
            face_image = frame[top:bottom, left:right]

            # Write the face image to the output folder
            output_path = os.path.join(output_folder, f"{os.path.basename(video_path)}_frame_{frame_count}.jpg")
            cv2.imwrite(output_path, face_image)

            # Increment frame count
            frame_count += 1
            frames_processed += 1
        
        if frames_processed == 5:
            break

    # Release the video capture object
    video_capture.release()

# Process videos in the first folder
for video_file in os.listdir(videos_folder_1):
    if video_file.endswith(".mp4"):
        video_path = os.path.join(videos_folder_1, video_file)
        extract_faces_from_video(video_path, output_folder1)

# Process videos in the second folder
for video_file in os.listdir(videos_folder_2):
    if video_file.endswith(".mp4"):
        video_path = os.path.join(videos_folder_2, video_file)
        extract_faces_from_video(video_path, output_folder2)

### EXTRACTING FACE CROPPED VIDEOS

In [ ]:
import cv2
import os
import glob
import face_recognition
from tqdm import tqdm

# Function to extract frames from a video
def extract_frames(video_path):
    frames = []
    vidObj = cv2.VideoCapture(video_path) 
    success = True
    while success:
        success, image = vidObj.read()
        if success:
            frames.append(image)
    return frames

# Function to detect faces in frames and save cropped face video clips
def create_face_videos(video_paths, output_dir):
    already_present_count = len(glob.glob(os.path.join(output_dir, '*.mp4')))
    print("Number of videos already present: ", already_present_count)
    
    for video_path in tqdm(video_paths):
        out_path = os.path.join(output_dir, os.path.basename(video_path))
        if os.path.exists(out_path):
            print("File already exists: ", out_path)
            continue
        
        frames = extract_frames(video_path)
        frame_count = len(frames)
        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112, 112))
        
        for idx, frame in enumerate(frames):
            if idx <= 150:  # Process only the first 150 frames
                faces = face_recognition.face_locations(frame)
                if faces:
                    top, right, bottom, left = faces[0]
                    face_frame = frame[top:bottom, left:right, :]
                    resized_face = cv2.resize(face_frame, (112, 112))
                    out.write(resized_face)
        
        out.release()

# Example usage
video_paths = glob.glob("/Users/karthiksagar/DeepFake-Detection/Celeb-DF/fake/*.mp4")
output_directory = '/Users/karthiksagar/DeepFake-Detection/Celeb-DF/facecropped/fake'
create_face_videos(video_paths, output_directory)


In [2]:
import cv2
import os
import glob
import face_recognition
from tqdm import tqdm

# Function to extract frames from a video
def extract_frames(video_path):
    frames = []
    vidObj = cv2.VideoCapture(video_path) 
    success = True
    while success:
        success, image = vidObj.read()
        if success:
            frames.append(image)
    return frames

# Function to detect faces in frames and save cropped face video clips
def create_face_videos(video_paths, output_dir):
    already_present_count = len(glob.glob(os.path.join(output_dir, '*.mp4')))
    print("Number of videos already present: ", already_present_count)
    
    for video_path in tqdm(video_paths):
        out_path = os.path.join(output_dir, os.path.basename(video_path))
        if os.path.exists(out_path):
            print("File already exists: ", out_path)
            continue
        
        frames = extract_frames(video_path)
        frame_count = len(frames)
        out = cv2.VideoWriter(out_path, cv2.VideoWriter_fourcc('M','J','P','G'), 30, (112, 112))
        
        for idx, frame in enumerate(frames):
            if idx <= 150:  # Process only the first 150 frames
                faces = face_recognition.face_locations(frame)
                if faces:
                    top, right, bottom, left = faces[0]
                    face_frame = frame[top:bottom, left:right, :]
                    resized_face = cv2.resize(face_frame, (112, 112))
                    out.write(resized_face)
        
        out.release()

# Example usage
video_paths = glob.glob("/Users/karthiksagar/DeepFake-Detection/Celeb-DF/real/*.mp4")
output_directory = '/Users/karthiksagar/DeepFake-Detection/Celeb-DF/facecropped/real'
create_face_videos(video_paths, output_directory)


Number of videos already present:  0


  0%|          | 0/405 [00:00<?, ?it/s]OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
  0%|          | 1/405 [00:23<2:39:54, 23.75s/it]OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
  0%|          | 2/405 [00:46<2:36:43, 23.33s/it]OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
  1%|          | 3/405 [01:10<2:37:27, 23.50s/it]OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part 14)'
OpenCV: FFMPEG: fallback to use tag 0x7634706d/'mp4v'
  1%|          | 4/405 [01:32<2:33:49, 23.02s/it]OpenCV: FFMPEG: tag 0x47504a4d/'MJPG' is not supported with codec id 7 and format 'mp4 / MP4 (MPEG-4 Part

KeyboardInterrupt: 